In [0]:
import numpy as np
import math 

In [0]:
# Run this cell to mount Google Drive.
# Do this in order to access files from collab 
from google.colab import drive
drive.mount('drive'); 

In [0]:
#arduino output was copied into a text file and place into google drive 
#using collab it can be read into a python array with numpy
textfile = "/content/drive/My Drive/1C Data/kf_test.txt"; 
read_in_array = np.loadtxt(textfile,skiprows=1,delimiter=','); 

#ardiuno output is measurements outputted on one line with a comma in between 
#measurements are outputted every dt or 10 ms  
#in between measurements there is a newline 
#measurement format: 
#lidar sensor front,lidar sensor right,gyroscope z angular velocity,gyroscope z in degrees
dn = read_in_array[:,0]
ln = read_in_array[:,1]
theta_dot = read_in_array[:,2]
theta = read_in_array[:,3]
#along with that PWM values are outputted at the very end 
pwm_left = read_in_array[:,4]
pwm_right = read_in_array[:,5]

In [0]:
def convert_pwm_to_omega(pwm):
  
  if((pwm>=85) and (pwm<=95)):
    omega = 0
  elif((pwm>95) and (pwm<125)):
    omega = 0.1 
  elif((pwm>75) and (pwm<85)):
    omega = -0.1 
  elif(pwm<=75):
    omega = -0.2 
  else: 
    omega = 0.2
  return omega

omega_left = []
for i in pwm_left:
  omega_left.append(convert_pwm_to_omega(i))

print(omega_left)

omega_right = []
for i in pwm_right:
  omega_right.append(convert_pwm_to_omega(i))

print(omega_right)

theta = theta*(math.pi)/180


In [0]:
#calculate state-space descrition matrices 

#constants used by matrices: 
#width of the robot from wheel to wheel 
w = 12
#radius of the wheel 
rw = 12 
#defining size of the environment 
Lx = 100 
Ly = 100 
#sampling rate dt for measurement 
dt = .010 #in ms of delay() 

#discrete for all equations 

def setF(omega_r, omega_l, theta, dt, F):
    F[0,2] = dt*(-1*rw/w)*math.sin(theta)*(omega_r+omega_l)
    F[1,2] = dt*(rw/w)*math.cos(theta)*(omega_r+omega_l)
    
    I = np.eye((4)).astype(float);
    F = F+1

    F[3,3]=0
    #all other entries stay as zeroes 

    #print(F)
    return F

def setG(theta, dt, G):
    G[0,0] = dt*(rw/w)*math.cos(theta) #row1
    G[0,1] = dt*(rw/w)*math.cos(theta)

    G[1,0] = dt*(rw/2)*math.sin(theta) #row2
    G[1,1] = dt*(rw/2)*math.sin(theta)

    G[2,0] = dt*(-rw/w) #row3
    G[2,1] = dt*(rw/w)

    #row4 is zeroes

    #print(G)
    return G

#this is the matrix for sensor model 
#uses trig in order to handle cases of car going straight vs. turning 
#Function is organized with variables in following order: 
#[a, b, c, 0,
# d, e, f, 0,
# 0, 0, 0, 0]
def setH(theta, x, y, H): 
    a1 = -1/math.cos(theta)
    a2 = 1/math.cos(theta+math.pi)
    H[0,0] = min(a1, a2)

    b1 = -1/math.sin(theta)
    b2 = 1/math.sin(theta+math.pi)
    H[0,1] = min(b1, b2)

    c1 = ((Lx-x)*math.tan(theta))/math.cos(theta)
    c2 = (x*math.tan(theta))/math.cos(theta)
    c3 = (-(Ly-y))/(math.tan(theta)*math.sin(theta))
    c4 = (-1)/(math.tan(theta+math.pi)*math.sin(theta+math.pi))
    H[0,2] = min(c1, c2, c3, c4)

    d1 = -1/math.sin(theta)
    d2 = 1/math.sin(theta+math.pi) 
    H[1,0] = min(d1, d2)

    e1 = -1/math.cos(theta)
    e2 = 1/math.cos(theta+math.pi)
    H[1,1] = min(e1, e2)

    f1 = (-(Lx-x))/(math.sin(theta)*math.tan(theta))
    f2 = (-x)/(math.sin(theta+math.pi)*math.tan(theta+math.pi))
    f3 = ((Ly-y)*(math.tan(theta+math.pi)))/(math.cos(theta+math.pi))
    f4 = (math.tan(theta)*y)/math.cos(theta)
    H[1,2] = min(f1,f2,f3,f4)

    return H; 



In [0]:
#defining the state using measurement 

class carMeasurement:
    def __init__(self):
        self.dn = 0 #front lidar sensor
        self.ln = 0 #side lidar sensor 
        self.gyroz = 0
        self.theta = 0 

class carState:
    def __init__(self):
        self.X =  np.zeros((4,1)).astype(float)
        self.P = np.zeros((4,4)).astype(float)
        self.measurement = carMeasurement()
    def set_state(self, X1, P1):
        self.X = X1
        self.P = P1
    def set_measurement(self, CM): 
        self.measurement = CM 


In [0]:
#Calculate the kalman filter 

#constants used by Kalman Filter for noise 
#Dynamics error 
#Qt=E[v vT]
Q = np.array([[0.2,0,0,0],
              [0,0.2,0,0],
              [0,0,0.2,0],
              [0,0,0,0.2]])
#Sensor Error
#w = [w1 w2 w3]^T 
#w1 = front lidar noise, w2 = right lidar noise, w3 = gyro noise 
#Rt=E[w wT]
R = np.array([[0.083, 0, 0],
              [0, 0.003, 0],
              [0, 0, 0.01]])

def kalman_filter(x_tplus, P_tplus, U, Z, F, G, H):
    #X(t+1)- = F(t)*X(t)+ + G(t)*U(t)
    #This is prediction of next state after taking measurement at current state
    FX = np.matmul(F, x_tplus)
    GU = np.matmul(G, U)
    x_tplus1minus = FX + GU 
   
    #Create covariance matrix P(t+1)- for the predicted state 
    FP = np.matmul(F,P_tplus)
    FPF = np.matmul(FP,np.transpose(F))
    #GQ = np.matmul(G,Q)
    #GPG = np.matmul(GP,np.transpose(G))
    P_tplus1minus = FPF +  Q
 
    #using the covariance matrix, kalman gain K(t) can be calculated 
    K = kalman_gain(P_tplus1minus, H)
  
    #with the kalman gain we can compute the measurement update 
    #this means using measurement z(t) to go from x(t)- to x(t)+ 
    #X(t)+ = X(t)- + K(t)*(Z(t) - H(t)*X(t)-)
    XH = np.matmul(H, x_tplus1minus)
    Z_XH = Z - XH
    K_Z_XH = np.matmul(K,Z_XH) 
    x_tplus1plus = x_tplus1minus + K_Z_XH

    #Need to also update the covariance matrix for the next state plus 
    KH = (1-np.matmul(K,H)) 
    P_tplus1plus = np.matmul(KH,P_tplus1minus)

    #Create the object for the new state and return 
    new_state = carState()
    new_state.set_state(x_tplus1plus, P_tplus1plus)
    return new_state

#kalman gain K(t) = P(t)-*H(t)T*(H(t)*P(t)-*H(t)T + R)-1 
#this is a helper to to the kalman filter to get measurement updates  
def kalman_gain(P_tplus1minus, H):
    HP = np.matmul(H, P_tplus1minus)
    HPH_R = np.matmul(HP, np.transpose(H)) + R
    HPH_R_inv = np.linalg.inv(HPH_R)

    H_HPH_R_inv = np.matmul(np.transpose(H), HPH_R_inv)
    PH_HPH_R_inv = np.matmul(P_tplus1minus, H_HPH_R_inv)
  
    K = PH_HPH_R_inv

    return K;

In [0]:
#Known initial state of car: 
#Car starts at rest 
#It is at the bottom left corner of the environment 
#It is 50cm away from the front and 50cm to the right 
#It is angled straightforward so 90 degrees from the right 0 line 
#It has no initial rotational velocity
def known_initial_loop(): 
    #known intials   
    X = np.array([[0],
                  [0],
                  [math.pi/2], #radians
                  [0]])
    P = np.zeros((4,4)).astype(float); 
    U = np.array([[-0.2],
                 [0.2]])
    Z = np.array([[40],
                  [40],
                  [0]])

    #create first state
    state = carState()
    #set first state with known initials 
    state.set_state(X, P)

    F = np.zeros((4,4)).astype(float)
    G = np.zeros((4,2)).astype(float)
    H = np.zeros((3,4)).astype(float)
    F = setF(0, 0, state.X[2], dt, F)
    G = setG(state.X[2], dt, G)
    H = setH(state.X[2], state.X[0], state.X[1], H)
    #continue to loop through remaining measurements taken 
    i = 0
    while(i < np.size(dn)):      
      Z[0] = dn[i] 
      Z[1] = ln[i]
      Z[2] = theta_dot[i] 
      U[0] = omega_left[i]
      U[1] = omega_right[i]
      state = kalman_filter(state.X, state.P, U, Z, F, G, H)
      F = setF(omega_left[i], omega_right[i], theta[i], dt, F)
      G = setG(theta[i], dt, G)
      H = setH(theta[i]), state.X[0], state.X[1], H)
      print(state.X)
      i+=1

known_initial_loop()

#Unknown intial state 
def unknown_initial_loop(): 
    #create first state
    state = carState()
    F = np.zeros((4,4)).astype(float)
    G = np.zeros((4,2)).astype(float)
    H = np.zeros((3,4)).astype(float)
    F = setF(0, 0, state.X[2], dt, F)
    G = setG(state.X[2], dt, G) 
    H = setH(state.X[2], state.X[0], state.X[1], H)
    #continue to loop through remaining measurements taken 
    i = 0
    while(i < np.size(dn)):      
      Z[0] = dn[i] 
      Z[1] = ln[i]
      Z[2] = theta_dot[i] 
      Z[3] = theta[i] 
      state = kalman_filter(state.X, state.P, U, Z, F, G, H)
      F = setF(0, 0, state.X[2], dt, F)
      G = setG(state.X[2], dt, G)
      H = setH(state.X[2], state.X[0], state.X[1], H)
      print(state.X)
      i+=1

